# Entry & Exit Signal

- Create separate notebook as 'poc.ipynb' notebook is too bloated.
- Create abstract class


In [2]:
import pandas as pd
from abc import ABC, abstractmethod
from datetime import datetime, date
import re
from collections import deque, Counter
from decimal import Decimal
import numpy as np
import yfinance as yf
from pydantic import BaseModel, Field, computed_field, field_validator
import math
from pathlib import Path
from zoneinfo import ZoneInfo
import calendar
import itertools
from omegaconf import OmegaConf 

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils
from config.variables import EntryType, CointCorrFn, STRUCT_MAPPING, EXIT_PRICE_MAPPING
from src.strategy.base.stock_trade import StockTrade

In [ ]:
results = utils.load_csv(
    "./data/results/2025-04-01/ziweichen_coint_5/trade_results.csv"
)
results

,ticker,coint_ticker,action,entry_date,entry_price,exit_date,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,UNH,MKTX,buy,2025-03-06,206.15,2025-03-07,208.94,1,2.79,0.013534,0.013534,1
1,UNH,MKTX,buy,2025-03-10,212.57,2025-03-14,212.38,4,-0.19,-0.000894,-0.000224,0
2,UNH,MKTX,buy,2025-03-19,216.75,2025-03-27,219.01,8,2.26,0.010427,0.001297,1
3,UNH,MKTX,buy,2025-03-20,217.82,2025-03-27,219.01,7,1.19,0.005463,0.000779,1
4,UNH,MKTX,buy,2025-03-25,213.42,2025-03-27,219.01,2,5.59,0.026192,0.013011,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,JPM,WTW,buy,2025-03-24,337.22,2025-03-27,340.88,3,3.66,0.010853,0.003605,1
1026,JPM,WTW,buy,2025-03-25,337.93,2025-03-27,340.88,2,2.95,0.00873,0.004356,1
1027,BAC,ABT,buy,2025-03-03,140.22,2025-03-05,137.71,2,-2.51,-0.0179,-0.00899,0
1028,BAC,ABT,buy,2025-03-13,129.89,2025-03-17,127.64,4,-2.25,-0.017322,-0.004359,0


In [12]:
neg = results.loc[results["win"] == 0, :]
neg

,ticker,coint_ticker,action,entry_date,entry_price,exit_date,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
1,UNH,MKTX,buy,2025-03-10,212.57,2025-03-14,212.38,4,-0.19,-0.000894,-0.000224,0
6,GOOGL,HPE,buy,2025-03-25,16.52,2025-03-26,16.07,1,-0.45,-0.02724,-0.02724,0
8,GOOGL,GOOG,buy,2025-03-25,172.79,2025-03-26,167.14,1,-5.65,-0.032699,-0.032699,0
9,BA,WBD,buy,2025-03-12,10.42,2025-03-13,9.89,1,-0.53,-0.050864,-0.050864,0
11,BA,WBD,buy,2025-03-17,10.48,2025-03-18,10.39,1,-0.09,-0.008588,-0.008588,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1011,CRM,HUM,buy,2025-03-06,265.76,2025-03-07,259.76,1,-6.0,-0.022577,-0.022577,0
1016,CRM,HUM,buy,2025-03-25,273.77,2025-03-27,270.38,2,-3.39,-0.012383,-0.006211,0
1017,CRM,HUM,buy,2025-03-26,270.48,2025-03-27,270.38,1,-0.1,-0.00037,-0.00037,0
1027,BAC,ABT,buy,2025-03-03,140.22,2025-03-05,137.71,2,-2.51,-0.0179,-0.00899,0


In [17]:
breakdown_neg = neg.groupby(by=["ticker", "coint_ticker"]).agg(
    {"percent_ret": ["min", "max", "mean", "median"]}
)
breakdown_neg

percent_ret                               
                            min        max      mean    median
ticker coint_ticker                                           
AMZN   ABT            -0.004717  -0.003471 -0.004094 -0.004094
       CAG            -0.008494  -0.008494 -0.008494 -0.008494
       CPAY           -0.017374  -0.001564 -0.007493 -0.003541
       DLR            -0.039167  -0.006803  -0.01977  -0.01334
       DLTR             -0.0076    -0.0076   -0.0076   -0.0076
...                         ...        ...       ...       ...
WMT    RCL            -0.035197  -0.022931 -0.028586 -0.028108
XOM    BDX            -0.014691  -0.006602 -0.011612 -0.013543
       ED             -0.005439  -0.005439 -0.005439 -0.005439
       GPN            -0.004504  -0.004504 -0.004504 -0.004504
       SWKS           -0.064616  -0.013246 -0.032636 -0.028215

[129 rows x 4 columns]

In [ ]:
breakdown_neg.columns = [
    "neg_ret_max",
    "neg_ret_min",
    "neg_ret_mean",
    "neg_ret_median",
]
breakdown_neg

neg_ret_max neg_ret_min neg_ret_mean neg_ret_median
ticker coint_ticker                                                    
AMZN   ABT            -0.004717   -0.003471    -0.004094      -0.004094
       CAG            -0.008494   -0.008494    -0.008494      -0.008494
       CPAY           -0.017374   -0.001564    -0.007493      -0.003541
       DLR            -0.039167   -0.006803     -0.01977       -0.01334
       DLTR             -0.0076     -0.0076      -0.0076        -0.0076
...                         ...         ...          ...            ...
WMT    RCL            -0.035197   -0.022931    -0.028586      -0.028108
XOM    BDX            -0.014691   -0.006602    -0.011612      -0.013543
       ED             -0.005439   -0.005439    -0.005439      -0.005439
       GPN            -0.004504   -0.004504    -0.004504      -0.004504
       SWKS           -0.064616   -0.013246    -0.032636      -0.028215

[129 rows x 4 columns]

In [20]:
df = utils.load_csv(
    "./data/results/2025-04-01/ziweichen_coint_5/breakdown_summary.csv",
    header=[0, 1],
    index_col=[0, 1],
)
df

entry_date             exit_date days_held                \
                           min        max        max       min max      mean   
ticker coint_ticker                                                            
AMZN   ABT          2025-03-20 2025-03-25 2025-03-26         1   2  1.333333   
       CAG          2025-03-20 2025-03-25 2025-03-26         1   2  1.333333   
       CPAY         2025-03-20 2025-03-25 2025-03-26         1   2  1.333333   
       DLR          2025-03-20 2025-03-25 2025-03-26         1   2  1.333333   
       DLTR         2025-03-20 2025-03-25 2025-03-26         1   2  1.333333   
...                        ...        ...        ...       ...  ..       ...   
XOM    EG           2025-03-11 2025-03-26 2025-03-27         1  16       9.3   
       GPN          2025-03-11 2025-03-26 2025-03-27         1  16       9.3   
       MTCH         2025-03-11 2025-03-26 2025-03-27         1  16       9.3   
       NOC          2025-03-11 2025-03-26 2025-03-27         1  16       9.3   
       SWKS         2025-03-11 2025-03-26 2025-03-27         1  16       9.3   

                           entry_price profit_loss         ...          \
                    median         sum         min    max  ...     sum   
ticker coint_ticker                                        ...           
AMZN   ABT             1.0      379.59        -0.6   1.01  ...   -0.03   
       CAG             1.0       76.98       -0.22   0.57  ...    0.54   
       CPAY            1.0     1069.43       -6.13  -0.56  ...   -7.96   
       DLR             1.0      458.32       -6.09  -1.03  ...   -9.14   
       DLTR            1.0      201.47       -0.53   2.16  ...     3.7   
...                    ...         ...         ...    ...  ...     ...   
XOM    EG              9.5     3564.61        1.35   21.7  ...  110.49   
       GPN             9.5      956.87       -0.45   7.54  ...   37.83   
       MTCH            9.5      310.27        0.01   1.76  ...    8.13   
       NOC             9.5     4918.75        6.68  38.72  ...  219.45   
       SWKS            9.5      685.59       -4.58  -0.89  ...  -22.59   

                     daily_ret                                    win      \
                           min        max       mean     median count sum   
ticker coint_ticker                                                         
AMZN   ABT           -0.003471   0.008041   0.000736  -0.002361     3   1   
       CAG           -0.008494   0.022485   0.005892   0.003685     3   2   
       CPAY          -0.017374  -0.000782  -0.007232  -0.003541     3   0   
       DLR           -0.019779  -0.006803  -0.013307   -0.01334     3   0   
       DLTR          -0.003807   0.033442   0.020155   0.030831     3   2   
...                        ...        ...        ...        ...   ...  ..   
XOM    EG             0.000669   0.007234   0.003872   0.003832    10  10   
       GPN           -0.004504   0.005647   0.003208   0.004079    10   9   
       MTCH           0.000314   0.009073    0.00336   0.002603    10  10   
       NOC            0.002448   0.013172   0.006042   0.005284    10  10   
       SWKS          -0.019231  -0.000952  -0.006162  -0.003966    10   0   

                    trading_period overall_percent_ret overall_daily_ret  
                                                                          
ticker coint_ticker                                                       
AMZN   ABT                       6           -0.000079         -0.000013  
       CAG                       6            0.007015          0.001166  
       CPAY                      6           -0.007443         -0.001244  
       DLR                       6           -0.019942         -0.003352  
       DLTR                      6            0.018365          0.003038  
...                            ...                 ...               ...  
XOM    EG                       16            0.030996           0.00191  
       GPN                      1

In [21]:
df.columns = ["_".join(col_tuple) for col_tuple in df.columns]
df

entry_date_min entry_date_max exit_date_max  \
ticker coint_ticker                                               
AMZN   ABT              2025-03-20     2025-03-25    2025-03-26   
       CAG              2025-03-20     2025-03-25    2025-03-26   
       CPAY             2025-03-20     2025-03-25    2025-03-26   
       DLR              2025-03-20     2025-03-25    2025-03-26   
       DLTR             2025-03-20     2025-03-25    2025-03-26   
...                            ...            ...           ...   
XOM    EG               2025-03-11     2025-03-26    2025-03-27   
       GPN              2025-03-11     2025-03-26    2025-03-27   
       MTCH             2025-03-11     2025-03-26    2025-03-27   
       NOC              2025-03-11     2025-03-26    2025-03-27   
       SWKS             2025-03-11     2025-03-26    2025-03-27   

                     days_held_min  days_held_max days_held_mean  \
ticker coint_ticker                                                
AMZN   ABT                       1              2       1.333333   
       CAG                       1              2       1.333333   
       CPAY                      1              2       1.333333   
       DLR                       1              2       1.333333   
       DLTR                      1              2       1.333333   
...                            ...            ...            ...   
XOM    EG                        1             16            9.3   
       GPN                       1             16            9.3   
       MTCH                      1             16            9.3   
       NOC                       1             16            9.3   
       SWKS                      1             16            9.3   

                    days_held_median entry_price_sum profit_loss_min  \
ticker coint_ticker                                                    
AMZN   ABT                       1.0          379.59            -0.6   
       CAG                       1.0           76.98           -0.22   
       CPAY                      1.0         1069.43           -6.13   
       DLR                       1.0          458.32           -6.09   
       DLTR                      1.0          201.47           -0.53   
...                              ...             ...             ...   
XOM    EG                        9.5         3564.61            1.35   
       GPN                       9.5          956.87           -0.45   
       MTCH                      9.5          310.27            0.01   
       NOC                       9.5         4918.75            6.68   
       SWKS                      9.5          685.59           -4.58   

                    profit_loss_max  ... profit_loss_sum daily_ret_min  \
ticker coint_ticker                  ...                                 
AMZN   ABT                     1.01  ...           -0.03     -0.003471   
       CAG                     0.57  ...            0.54     -0.008494   
       CPAY                   -0.56  ...           -7.96     -0.017374   
       DLR                    -1.03  ...           -9.14     -0.019779   
       DLTR                    2.16  ...             3.7     -0.003807   
...                             ...  ...             ...           ...   
XOM    EG                      21.7  ...          110.49      0.000669   
       GPN                     7.54  ...           37.83     -0.004504   
       MTCH                    1.76  ...            8.13      0.000314   
       NOC                    38.72  ...          219.45      0.002448   
       SWKS                   -0.89  ...          -22.59     -0.019231   

                    daily_ret_max daily_ret_mean daily_ret_median win_count  \
ticker coint_ticker                                                           
AMZN   ABT               0.008041       0.000736        -0.002361         3   
       CAG               0.022485       0.005892         0.003685         3   
       CPAY             -0.000782      -0.00723

In [22]:
df.join(breakdown_neg)

entry_date_min entry_date_max exit_date_max  \
ticker coint_ticker                                               
AMZN   ABT              2025-03-20     2025-03-25    2025-03-26   
       CAG              2025-03-20     2025-03-25    2025-03-26   
       CPAY             2025-03-20     2025-03-25    2025-03-26   
       DLR              2025-03-20     2025-03-25    2025-03-26   
       DLTR             2025-03-20     2025-03-25    2025-03-26   
...                            ...            ...           ...   
XOM    EG               2025-03-11     2025-03-26    2025-03-27   
       GPN              2025-03-11     2025-03-26    2025-03-27   
       MTCH             2025-03-11     2025-03-26    2025-03-27   
       NOC              2025-03-11     2025-03-26    2025-03-27   
       SWKS             2025-03-11     2025-03-26    2025-03-27   

                     days_held_min  days_held_max days_held_mean  \
ticker coint_ticker                                                
AMZN   ABT                       1              2       1.333333   
       CAG                       1              2       1.333333   
       CPAY                      1              2       1.333333   
       DLR                       1              2       1.333333   
       DLTR                      1              2       1.333333   
...                            ...            ...            ...   
XOM    EG                        1             16            9.3   
       GPN                       1             16            9.3   
       MTCH                      1             16            9.3   
       NOC                       1             16            9.3   
       SWKS                      1             16            9.3   

                    days_held_median entry_price_sum profit_loss_min  \
ticker coint_ticker                                                    
AMZN   ABT                       1.0          379.59            -0.6   
       CAG                       1.0           76.98           -0.22   
       CPAY                      1.0         1069.43           -6.13   
       DLR                       1.0          458.32           -6.09   
       DLTR                      1.0          201.47           -0.53   
...                              ...             ...             ...   
XOM    EG                        9.5         3564.61            1.35   
       GPN                       9.5          956.87           -0.45   
       MTCH                      9.5          310.27            0.01   
       NOC                       9.5         4918.75            6.68   
       SWKS                      9.5          685.59           -4.58   

                    profit_loss_max  ... daily_ret_median win_count win_sum  \
ticker coint_ticker                  ...                                      
AMZN   ABT                     1.01  ...        -0.002361         3       1   
       CAG                     0.57  ...         0.003685         3       2   
       CPAY                   -0.56  ...        -0.003541         3       0   
       DLR                    -1.03  ...         -0.01334         3       0   
       DLTR                    2.16  ...         0.030831         3       2   
...                             ...  ...              ...       ...     ...   
XOM    EG                      21.7  ...         0.003832        10      10   
       GPN                     7.54  ...         0.004079        10       9   
       MTCH                    1.76  ...         0.002603        10      10   
       NOC                    38.72  ...         0.005284        10      10   
       SWKS                   -0.89  ...        -0.003966        10       0   

                    trading_period_ overall_percent_ret_ overall_daily_ret_  \
ticker coint_ticker                                                           
AMZN   ABT                        6            -0.000079          -0.000013   
       CAG                        6             0.007015     

In [24]:
df.index.names

FrozenList(['ticker', 'coint_ticker'])

# Table of Contents

1. [GenTrades](#gentrades)


In [14]:
trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-01",
    entry_action="buy",
    entry_lots=10,
    entry_price=200,
)
trade1 = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=195,
)

open_trades = deque([trade, trade1])
class_name = STRUCT_MAPPING.get("multiple")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-4-4", "buy", 192)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [19]:
utils.display_open_trades(open_trades)

open_trades : 
[
   ticker: 'AAPL', ent_dt: '2025-04-01', ent_act: 'buy', ent_lots: 10, ent_price: 200, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-02', ent_act: 'buy', ent_lots: 10, ent_price: 195, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-04', ent_act: 'buy', ent_lots: 20, ent_price: 192, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
]



In [140]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "sell", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "sell", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "sell", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='sell', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='sell', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='sell', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [50]:
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [51]:
class_name = STRUCT_MAPPING.get("half_lifo")
exit_method = utils.get_class_instance(class_name, "./src/strategy/base/exit_struct.py")
open_trades, completed_trades = exit_method.close_pos(
    open_trades, "2025-04-10", "buy", 180
)
open_trades

Validation Error : 1 validation error for StockTrade
exit_action
  Value error, Entry action cannot be the same as exit action except when both are 'wait'. [type=value_error, input_value='buy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [131]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "buy", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "buy", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "buy", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='buy', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [141]:
class_name = EXIT_PRICE_MAPPING.get("percent_loss")
stop_price = utils.get_class_instance(
    class_name, "./src/strategy/base/cal_exit_price.py", percent_loss=0.1
)
stop_price.cal_exit_price(open_trades)

Decimal('212.61')

In [30]:
pos = "long"
trade1 = (10, 141.87)
trade2 = (10, 141.97)
trade3 = (10, 143.25)
percent_loss = 0.001
trades = [
    trade1,
    trade2,
    trade3,
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)
# stop_loss = [trade[1] * (1-percent_loss) if pos == "long" else trade[1] * (1+percent_loss) for trade in trades]
# stop_loss = [Decimal(str(round(loss, 2))) for loss in stop_loss]
# stop_loss

# (10 * 200 + 10 * 195 + 20 * 192 ) * (1-percent_loss)/ 40

stop_price = (
    total_invest * (1 - percent_loss) / total_lots
    if pos == "long"
    else total_invest * (1 + percent_loss) / total_lots
)
stop_price

142.22096999999997

In [32]:
142.9 * (1 - percent_loss)

142.7571

# [GenTrades](#table-of-contents)


In [2]:
date = "2025-04-01"
model = "ziweichen_coint_5"
ticker_pair = "NFLX_EA"

results_dir = f"./data/results/{date}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [3]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
df

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3.0
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3.0
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4.0
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5.0
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3.0
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5.0
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4.0
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3.0
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3.5
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3.0


In [134]:
entry_type = "long_only"
# entry_type = "short_only"
# entry_type = "long_or_short"

entry = utils.get_class_instance(
    "SentiEntry",
    "./src/strategy/entry/senti_entry.py",
    entry_type=entry_type,
)
exit = utils.get_class_instance(
    "SentiExit",
    "./src/strategy/exit/senti_exit.py",
    entry_type=entry_type,
)

df_signal = entry.gen_entry_signal(df)
df_signal = exit.gen_exit_signal(df_signal)
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [165]:
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [166]:
trades = utils.get_class_instance(
    "SentiTrades",
    "./src/strategy/trade/senti_trades.py",
    entry_struct="multiple",
    exit_struct="half_lifo",
    num_lots=10,
    percent_loss=0.01,
    exit_method="nearest_loss",
    monitor_close=False,
)

df_trades, df_pa = trades.gen_trades(df_signal)
df_trades

idx : 0
dt : 2025-03-03 16:00:00-05:00
close : 129.23
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 1
dt : 2025-03-04 16:00:00-05:00
close : 131.82
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 2
dt : 2025-03-05 16:00:00-05:00
close : 134.05
ent_sig : buy
ex_sig : wait
net_pos : 0
net_pos after update : 10
len(self.open_trades) : 1
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_lots: 0, ex_price: None
   },
]

idx : 3
dt : 2025-03-06 16:00:00-05:00
close : 136.79
ent_sig : buy
ex_sig : wait
net_pos : 10
stop_price [long] : 132.71 -> monitor low [current: 133.37]
net_pos after update : 20
len(self.open_trades) : 2
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,NFLX,EA,2025-03-05 16:00:00-05:00,buy,10,134.05,2025-03-11 16:00:00-04:00,sell,10,139.03,5,4.98,0.037150,0.007322,1
1,NFLX,EA,2025-03-06 16:00:00-05:00,buy,10,136.79,2025-03-11 16:00:00-04:00,sell,10,139.03,4,2.24,0.016375,0.004069,1
2,NFLX,EA,2025-03-10 16:00:00-04:00,buy,10,140.43,2025-03-11 16:00:00-04:00,sell,10,139.03,1,-1.40,-0.009969,-0.009969,0
3,NFLX,EA,2025-03-11 16:00:00-04:00,buy,10,137.88,2025-03-12 16:00:00-04:00,sell,10,136.50,1,-1.38,-0.010009,-0.010009,0
4,NFLX,EA,2025-03-17 16:00:00-04:00,buy,10,142.9,2025-03-20 16:00:00-04:00,sell,10,141.47,3,-1.43,-0.010007,-0.003347,0
5,NFLX,EA,2025-03-18 16:00:00-04:00,buy,10,141.87,2025-03-20 16:00:00-04:00,sell,10,141.47,2,-0.40,-0.002819,-0.001410,0
6,NFLX,EA,2025-03-19 16:00:00-04:00,buy,10,141.97,2025-03-20 16:00:00-04:00,sell,10,141.47,1,-0.50,-0.003522,-0.003522,0
7,NFLX,EA,2025-03-20 16:00:00-04:00,buy,5,143.25,2025-03-21 16:00:00-04:00,sell,5,144.3,1,1.05,0.007330,0.007330,1
8,NFLX,EA,2025-03-25 16:00:00-04:00,buy,10,144.5,2025-03-27 16:00:00-04:00,sell,10,145.3,2,0.8,0.005536,0.002764,1
9,NFLX,EA,2025-03-24 16:00:00-04:00,buy,3,144.55,2025-03-27 16:00:00-04:00,sell,3,145.3,3,0.75,0.005189,0.001727,1


In [167]:
df_pa

,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal,stop_price,triggered
date,,,,,,,,,,,,,,
2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait,NaN,NaN
2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait,NaN,NaN
2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait,NaN,NaN
2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait,132.71,0
2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait,135.42,0
2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait,135.42,0
2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait,139.03,1
2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait,136.50,1
2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait,NaN,NaN


In [107]:
pos = "long"
percent_loss = 0.01
trades = [
    # (10, 134.05),
    # (5, 136.79),
    # (3, 140.43),
    (10, 142.9),
    (10, 141.87),
    (10, 141.97),
    # (2, 137.88),
    # (10, 144.5),
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)

# stop_price = (
#     total_invest * (1 - percent_loss) / total_lots
#     if pos == "long"
#     else total_invest * (1 + percent_loss) / total_lots
# )
# stop_price

stop_list = [
    trade[1] * (1 - percent_loss) if pos == "long" else trade[1] * (1 + percent_loss)
    for trade in trades
]
stop_list

[141.471, 140.4513, 140.5503]

In [103]:
134.05 * (1 - percent_loss)

132.70950000000002

In [53]:
for ent_type, ent, ex in itertools.product(
    ["long_only", "short_only"],
    ["multiple", "multiple_half"],
    ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"],
):
    print(ent_type, ent, ex)

long_only multiple fifo
long_only multiple lifo
long_only multiple half_fifo
long_only multiple half_lifo
long_only multiple take_all
long_only multiple_half fifo
long_only multiple_half lifo
long_only multiple_half half_fifo
long_only multiple_half half_lifo
long_only multiple_half take_all
short_only multiple fifo
short_only multiple lifo
short_only multiple half_fifo
short_only multiple half_lifo
short_only multiple take_all
short_only multiple_half fifo
short_only multiple_half lifo
short_only multiple_half half_fifo
short_only multiple_half half_lifo
short_only multiple_half take_all


In [54]:
a = (
    list(
        itertools.product(
            ["long_only", "short_only"],
            ["multiple", "multiple_half"],
            ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"],
        )
    )
    + list(
        itertools.product(
            ["long_only", "short_only"], ["single"], ["fifo", "half_fifo"]
        )
    )
    + list(
        itertools.product(
            ["long_or_short"], ["multiple", "multiple_half", "single"], ["take_all"]
        )
    )
)
a

[('long_only', 'multiple', 'fifo'),
 ('long_only', 'multiple', 'lifo'),
 ('long_only', 'multiple', 'half_fifo'),
 ('long_only', 'multiple', 'half_lifo'),
 ('long_only', 'multiple', 'take_all'),
 ('long_only', 'multiple_half', 'fifo'),
 ('long_only', 'multiple_half', 'lifo'),
 ('long_only', 'multiple_half', 'half_fifo'),
 ('long_only', 'multiple_half', 'half_lifo'),
 ('long_only', 'multiple_half', 'take_all'),
 ('short_only', 'multiple', 'fifo'),
 ('short_only', 'multiple', 'lifo'),
 ('short_only', 'multiple', 'half_fifo'),
 ('short_only', 'multiple', 'half_lifo'),
 ('short_only', 'multiple', 'take_all'),
 ('short_only', 'multiple_half', 'fifo'),
 ('short_only', 'multiple_half', 'lifo'),
 ('short_only', 'multiple_half', 'half_fifo'),
 ('short_only', 'multiple_half', 'half_lifo'),
 ('short_only', 'multiple_half', 'take_all'),
 ('long_only', 'single', 'fifo'),
 ('long_only', 'single', 'half_fifo'),
 ('short_only', 'single', 'fifo'),
 ('short_only', 'single', 'half_fifo'),
 ('long_or_short

In [55]:
len(a)

27